In [1]:
from pathlib import Path
import requests

import pandas as pd


In [2]:
def download_file(year: int, month: int) -> Path:
    """
    D

    """
    URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"

    resp = requests.get(URL)

    # check to see if get request was sucessful
    if resp.status_code == 200:

        path = f"../data/raw/rides_{year}-{month:02d}.parquet"
        with open(path, 'wb') as f:
            f.write(resp.content)
        
        return path
    else:
        raise Exception(f'{URL} is not available')


In [3]:
download_file(year=2022, month=1)

'../data/raw/rides_2022-01.parquet'

In [4]:
rides = pd.read_parquet('../data/raw/rides_2022-01.parquet')
rides.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [7]:
columns = ['tpep_pickup_datetime', 'PULocationID' ]
rides = rides[columns]
rides.rename(columns={
    'tpep_pickup_datetime':'pickup_datetime', 
    'PULocationID':'pickup_location_id',
}, inplace=True)
rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [8]:
rides['pickup_datetime'].describe()

count                       2463931
mean     2022-01-17 01:19:51.689724
min             2008-12-31 22:23:09
25%             2022-01-09 15:37:41
50%             2022-01-17 12:11:45
75%      2022-01-24 13:49:37.500000
max             2022-05-18 20:41:57
Name: pickup_datetime, dtype: object

In [10]:
date_filter_mask = (rides.pickup_datetime >= '2022-01-01') & (rides.pickup_datetime < '2022-02-01')
rides = rides[date_filter_mask]
rides.pickup_datetime.describe()

count                       2463879
mean     2022-01-17 01:58:40.393674
min             2022-01-01 00:00:08
25%             2022-01-09 15:37:56
50%             2022-01-17 12:11:54
75%             2022-01-24 13:49:37
max             2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

In [12]:
rides.pickup_location_id.isna().sum()

np.int64(0)

In [13]:
rides.to_parquet('../data/transformed/validated_rides_2022_01.parquet')